# Rekurrens neurális hálózatok

Az előző modulban a szövegek gazdag szemantikai reprezentációit tárgyaltuk. Az általunk használt architektúra a mondatokban szereplő szavak összesített jelentését ragadja meg, de nem veszi figyelembe a szavak **sorrendjét**, mivel az embeddingeket követő aggregációs művelet eltávolítja ezt az információt az eredeti szövegből. Mivel ezek a modellek nem képesek a szórendet reprezentálni, nem tudnak megoldani összetettebb vagy kétértelmű feladatokat, például szövegalkotást vagy kérdésmegértést.

Ahhoz, hogy egy szövegszekvencia jelentését megragadjuk, egy **rekurrens neurális hálózatnak** (angolul recurrent neural network, RNN) nevezett neurális hálózati architektúrát fogunk használni. Az RNN használatakor a mondatot egyesével, tokenenként vezetjük át a hálózaton, amely minden lépésben előállít egy **állapotot**, amit aztán a következő tokennel együtt ismét átadunk a hálózatnak.

![Kép, amely egy rekurrens neurális hálózat generálását mutatja.](../../../../../translated_images/hu/rnn.27f5c29c53d727b5.webp)

A tokenekből álló bemeneti szekvencia $X_0,\dots,X_n$ alapján az RNN egy neurális hálózati blokkokból álló szekvenciát hoz létre, és ezt a szekvenciát végig, visszaterjesztéses tanulással (backpropagation) tanítja. Minden hálózati blokk egy $(X_i,S_i)$ párt kap bemenetként, és eredményként előállítja $S_{i+1}$-et. A végső állapot $S_n$ vagy a kimenet $Y_n$ egy lineáris osztályozóba kerül, amely előállítja az eredményt. Az összes hálózati blokk ugyanazokat a súlyokat osztja meg, és egyetlen visszaterjesztési lépés során tanulják meg azokat.

> A fenti ábra a rekurrens neurális hálózatot kibontott formában (bal oldalon), illetve kompaktabb, rekurrens reprezentációban (jobb oldalon) mutatja. Fontos megérteni, hogy az összes RNN cella ugyanazokat a **megosztható súlyokat** használja.

Mivel az állapotvektorok $S_0,\dots,S_n$ végighaladnak a hálózaton, az RNN képes megtanulni a szavak közötti szekvenciális függőségeket. Például, ha a *nem* szó megjelenik valahol a szekvenciában, a hálózat megtanulhatja bizonyos elemek tagadását az állapotvektoron belül.

Egy RNN cellán belül két súlymátrix található: $W_H$ és $W_I$, valamint egy bias $b$. Minden RNN lépésben, adott $X_i$ bemenet és $S_i$ bemeneti állapot esetén a kimeneti állapotot az alábbi képlet alapján számítjuk ki: $S_{i+1} = f(W_H\times S_i + W_I\times X_i+b)$, ahol $f$ egy aktivációs függvény (gyakran $\tanh$).

> Olyan problémák esetén, mint például a szövegalkotás (amit a következő egységben tárgyalunk) vagy a gépi fordítás, minden RNN lépésnél szeretnénk valamilyen kimeneti értéket is kapni. Ebben az esetben van egy másik mátrix is, $W_O$, és a kimenetet az alábbi módon számítjuk ki: $Y_i=f(W_O\times S_i+b_O)$.

Nézzük meg, hogyan segíthetnek a rekurrens neurális hálózatok a hírek adatbázisának osztályozásában.

> A sandbox környezetben az alábbi cellát kell futtatnunk, hogy biztosítsuk a szükséges könyvtár telepítését és az adatok előzetes letöltését. Ha helyben futtatja a kódot, ezt a cellát kihagyhatja.


In [1]:
import sys
!{sys.executable} -m pip install --quiet tensorflow_datasets==4.4.0
!cd ~ && wget -q -O - https://mslearntensorflowlp.blob.core.windows.net/data/tfds-ag-news.tgz | tar xz

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

# We are going to be training pretty large models. In order not to face errors, we need
# to set tensorflow option to grow GPU memory allocation when required
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

ds_train, ds_test = tfds.load('ag_news_subset').values()

Amikor nagy modelleket tanítunk, a GPU memória kiosztása problémát jelenthet. Emellett szükség lehet különböző minibatch méretek kipróbálására, hogy az adatok beleférjenek a GPU memóriába, ugyanakkor a tanítás elég gyors legyen. Ha ezt a kódot saját GPU gépen futtatod, kísérletezhetsz a minibatch méret beállításával a tanítás felgyorsítása érdekében.

> **Note**: Bizonyos NVidia driver verziókról ismert, hogy nem szabadítják fel a memóriát a modell tanítása után. Ebben a jegyzetfüzetben több példát futtatunk, ami bizonyos beállításoknál memória kimerüléséhez vezethet, különösen akkor, ha saját kísérleteket végzel ugyanazon jegyzetfüzet részeként. Ha furcsa hibákba ütközöl a modell tanításának megkezdésekor, érdemes lehet újraindítani a jegyzetfüzet kernelét.


In [3]:
batch_size = 16
embed_size = 64

## Egyszerű RNN osztályozó

Egy egyszerű RNN esetében minden rekurrens egység egy egyszerű lineáris hálózat, amely bemeneti vektort és állapotvektort fogad, majd egy új állapotvektort hoz létre. Keras-ban ezt a `SimpleRNN` réteg képviseli.

Bár közvetlenül átadhatunk egy-egy forró kódolású tokeneket az RNN rétegnek, ez nem jó ötlet a magas dimenzionalitásuk miatt. Ezért egy beágyazási réteget fogunk használni, hogy csökkentsük a szavak vektorainak dimenzionalitását, majd egy RNN réteget, végül pedig egy `Dense` osztályozót.

> **Megjegyzés**: Olyan esetekben, amikor a dimenzionalitás nem olyan magas, például karakter szintű tokenizálásnál, érdemes lehet az egy-egy forró kódolású tokeneket közvetlenül az RNN cellába továbbítani.


In [4]:
vocab_size = 20000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size,
    input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 64)          1280000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 16)                1296      
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
Total params: 1,281,364
Trainable params: 1,281,364
Non-trainable params: 0
_________________________________________________________________


> **Megjegyzés:** Itt egy nem tanított beágyazási réteget használunk az egyszerűség kedvéért, de jobb eredmények érdekében használhatunk egy előre betanított beágyazási réteget a Word2Vec segítségével, ahogy az előző egységben leírtuk. Jó gyakorlat lenne, ha ezt a kódot úgy alakítanád át, hogy előre betanított beágyazásokat használjon.

Most tanítsuk be az RNN-t. Az RNN-ek általában elég nehezen taníthatók, mivel amikor az RNN cellákat kibontjuk a szekvencia hosszának megfelelően, a visszaterjesztésben részt vevő rétegek száma jelentősen megnő. Ezért kisebb tanulási rátát kell választanunk, és nagyobb adathalmazon kell tanítani a hálózatot, hogy jó eredményeket érjünk el. Ez elég sok időt vehet igénybe, ezért előnyös GPU-t használni.

A folyamat felgyorsítása érdekében csak a hírek címein fogjuk tanítani az RNN modellt, kihagyva a leírást. Kipróbálhatod a leírással való tanítást, és megnézheted, hogy sikerül-e a modellt betanítani.


In [5]:
def extract_title(x):
    return x['title']

def tupelize_title(x):
    return (extract_title(x),x['label'])

print('Training vectorizer')
vectorizer.adapt(ds_train.take(2000).map(extract_title))

Training vectorizer


In [6]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize_title).batch(batch_size),validation_data=ds_test.map(tupelize_title).batch(batch_size))

7500/7500 [==============================] - 82s 11ms/step - loss: 0.6629 - acc: 0.7623 - val_loss: 0.5559 - val_acc: 0.7995


> **Megjegyzés**: a pontosság valószínűleg alacsonyabb lesz, mivel csak hírcímeken képezzük a modelt.


## A változó hosszúságú szekvenciák újragondolása

Ne feledd, hogy a `TextVectorization` réteg automatikusan kitölti a változó hosszúságú szekvenciákat egy minibatch-ben kitöltő tokenekkel. Kiderült, hogy ezek a tokenek is részt vesznek a tanításban, és megnehezíthetik a modell konvergenciáját.

Számos megközelítést alkalmazhatunk a kitöltés mennyiségének minimalizálására. Az egyik lehetőség, hogy az adathalmazt szekvenciahossz szerint rendezzük, és az összes szekvenciát méret szerint csoportosítjuk. Ez a `tf.data.experimental.bucket_by_sequence_length` függvénnyel valósítható meg (lásd [dokumentáció](https://www.tensorflow.org/api_docs/python/tf/data/experimental/bucket_by_sequence_length)).

Egy másik megközelítés a **maszkolás** használata. A Keras-ban bizonyos rétegek támogatják az olyan kiegészítő bemeneteket, amelyek megmutatják, hogy mely tokeneket kell figyelembe venni a tanítás során. A maszkolás beépítéséhez a modellünkbe vagy egy külön `Masking` réteget kell hozzáadnunk ([dokumentáció](https://keras.io/api/layers/core_layers/masking/)), vagy meg kell adnunk a `mask_zero=True` paramétert az `Embedding` rétegünkben.

> **Note**: Ez a tanítás körülbelül 5 percet vesz igénybe, hogy az egész adathalmazon egy epochot lefuttasson. Nyugodtan megszakíthatod a tanítást bármikor, ha elfogy a türelmed. Amit még tehetsz, hogy korlátozod a tanításhoz használt adatok mennyiségét, például `.take(...)` kifejezést hozzáadva a `ds_train` és `ds_test` adathalmazok után.


In [7]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size,embed_size,mask_zero=True),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

7500/7500 [==============================] - 371s 49ms/step - loss: 0.5401 - acc: 0.8079 - val_loss: 0.3780 - val_acc: 0.8822


Most, hogy maszkolást használunk, az egész címek és leírások adatállományán tudjuk tanítani a modellt.

> **Megjegyzés**: Észrevetted, hogy eddig a hírek címein tanított vektorizálót használtuk, nem pedig a cikk teljes szövegét? Ez esetleg azt eredményezheti, hogy néhány token figyelmen kívül marad, ezért jobb lenne újra tanítani a vektorizálót. Azonban ennek valószínűleg csak nagyon csekély hatása lenne, így az egyszerűség kedvéért maradunk a korábban betanított vektorizálónál.


## LSTM: Hosszú távú memória

Az RNN-ek egyik fő problémája a **eltűnő gradiens**. Az RNN-ek elég hosszúak lehetnek, és nehézséget okozhat a gradiens visszavezetése egészen a hálózat első rétegéig a visszaterjesztés során. Amikor ez megtörténik, a hálózat nem tud tanulni távoli tokenek közötti kapcsolatokat. Ennek a problémának az elkerülésére egy megoldás az **explicit állapotkezelés** bevezetése **kapuk** használatával. A két leggyakoribb architektúra, amely kapukat alkalmaz, a **hosszú távú memória** (LSTM) és a **kapuzott reléegység** (GRU). Itt az LSTM-eket fogjuk tárgyalni.

![Kép, amely egy hosszú távú memória cella példáját mutatja](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

Az LSTM hálózat felépítése hasonló az RNN-hez, de két állapotot adunk át rétegről rétegre: az aktuális állapotot $c$, és a rejtett vektort $h$. Minden egységnél a rejtett vektor $h_{t-1}$ kombinálódik a bemenettel $x_t$, és együtt irányítják, hogy mi történik az állapottal $c_t$ és a kimenettel $h_{t}$ **kapukon** keresztül. Minden kapunak szigmoid aktivációja van (kimenet tartománya $[0,1]$), amely bitmaszkként értelmezhető, amikor megszorozzuk az állapotvektorral. Az LSTM-eknek a következő kapui vannak (a fenti képen balról jobbra):
* **felejtő kapu**, amely meghatározza, hogy az $c_{t-1}$ vektor mely komponenseit kell elfelejtenünk, és melyeket kell továbbadnunk.
* **bemeneti kapu**, amely meghatározza, hogy mennyi információt kell a bemeneti vektorból és az előző rejtett vektorból beépíteni az állapotvektorba.
* **kimeneti kapu**, amely az új állapotvektort veszi, és eldönti, hogy annak mely komponenseit használjuk az új rejtett vektor $h_t$ előállításához.

Az állapot $c$ komponensei zászlókként értelmezhetők, amelyeket be- és kikapcsolhatunk. Például, amikor a *Alice* nevet találjuk a sorozatban, feltételezzük, hogy egy nőre utal, és felállítjuk az állapotban azt a zászlót, amely azt jelzi, hogy nőnemű főnév van a mondatban. Amikor később találkozunk az *és Tom* szavakkal, felállítjuk azt a zászlót, amely azt jelzi, hogy többes számú főnév van. Így az állapot manipulálásával nyomon követhetjük a mondat nyelvtani tulajdonságait.

> **Note**: Itt egy remek forrás az LSTM-ek belső működésének megértéséhez: [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) Christopher Olah-tól.

Bár az LSTM cella belső szerkezete bonyolultnak tűnhet, a Keras elrejti ezt a megvalósítást az `LSTM` rétegben, így az egyetlen dolog, amit az előző példában tennünk kell, az az, hogy lecseréljük a visszatérő réteget:


In [8]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.LSTM(8),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(8),validation_data=ds_test.map(tupelize).batch(8))

15000/15000 [==============================] - 188s 13ms/step - loss: 0.5692 - acc: 0.7916 - val_loss: 0.3441 - val_acc: 0.8870


## Kétirányú és többrétegű RNN-ek

Az eddigi példáinkban a rekurrens hálózatok a szekvencia elejétől a végéig működtek. Ez számunkra természetesnek tűnik, mivel ugyanazt az irányt követi, ahogy olvasunk vagy beszédet hallgatunk. Azonban olyan helyzetekben, ahol az input szekvenciához véletlenszerű hozzáférésre van szükség, logikusabb, ha a rekurrens számítást mindkét irányban futtatjuk. Azokat az RNN-eket, amelyek lehetővé teszik a számítást mindkét irányban, **kétirányú** RNN-eknek nevezzük, és létrehozhatók úgy, hogy a rekurrens réteget egy speciális `Bidirectional` réteggel csomagoljuk be.

> **Note**: A `Bidirectional` réteg két példányt készít a benne lévő rétegből, és az egyik példány `go_backwards` tulajdonságát `True` értékre állítja, így az a szekvencia mentén ellentétes irányba halad.

A rekurrens hálózatok, legyenek egyirányúak vagy kétirányúak, mintákat ragadnak meg egy szekvenciában, és ezeket állapotvektorokba tárolják vagy kimenetként adják vissza. Akárcsak a konvolúciós hálózatok esetében, építhetünk egy másik rekurrens réteget az első után, hogy magasabb szintű mintákat ragadjunk meg, amelyeket az első réteg által kinyert alacsonyabb szintű mintákból építünk fel. Ez vezet el minket a **többrétegű RNN** fogalmához, amely két vagy több rekurrens hálózatból áll, ahol az előző réteg kimenete bemenetként kerül a következő réteghez.

![Kép egy többrétegű hosszú-rövid távú memória RNN-ről](../../../../../translated_images/hu/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Kép [ebből a nagyszerű bejegyzésből](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) Fernando López tollából.*

A Keras megkönnyíti ezeknek a hálózatoknak a létrehozását, mivel csak több rekurrens réteget kell hozzáadni a modellhez. Az utolsó rétegen kívül minden rétegnél meg kell adni a `return_sequences=True` paramétert, mivel szükségünk van arra, hogy a réteg az összes köztes állapotot visszaadja, ne csak a rekurrens számítás végső állapotát.

Építsünk egy két rétegű kétirányú LSTM-et a klasszifikációs problémánkhoz.

> **Note** Ez a kód ismét meglehetősen hosszú időt vesz igénybe, de ez adja a legmagasabb pontosságot, amit eddig láttunk. Talán érdemes várni és megnézni az eredményt.


In [9]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, 128, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64,return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),    
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

5045/7500 [===================>..........] - ETA: 2:33 - loss: 0.3709 - acc: 0.8706

## RNN-ek más feladatokra

Eddig az RNN-eket szövegszekvenciák osztályozására használtuk. Azonban sok más feladatot is képesek kezelni, például szöveg generálását és gépi fordítást — ezeket a feladatokat a következő egységben fogjuk megvizsgálni.



---

**Felelősségkizárás**:  
Ez a dokumentum az [Co-op Translator](https://github.com/Azure/co-op-translator) AI fordítási szolgáltatás segítségével készült. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt a professzionális, emberi fordítás igénybevétele. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy téves értelmezésekért.
